In [1]:
#import necessary libraries
import tensorflow  as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

from tensorflow.examples.tutorials.mnist import input_data #load the data

c:\users\madhivarman\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#load the dataset
mnist = input_data.read_data_sets('../../MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as offi

In [4]:
mb_size = 64  #batch
z_dim = 100 #number of dimensions
X_dim = mnist.train.images.shape[1] #number of data
y_dim = mnist.train.labels.shape[1] #number of labels
h_dim = 120
c=0
lr = 1e-3   #learning rate

In [5]:
print(X_dim,y_dim)

784 10


In [22]:
def plot(samples):
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace=0.05,hspace=0.05)
    
    for i,sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28,28),cmap='Greys_r')
        
    return fig

In [7]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim/2.)
    
    return tf.random_normal(shape=size,stddev=xavier_stddev)

In [18]:
# Q(z|x)

X = tf.placeholder(tf.float32,shape=[None,X_dim],name='X') #(?,784)
z= tf.placeholder(tf.float32,shape=[None,z_dim])

Q_w1 = tf.Variable(xavier_init([X_dim,h_dim]),name='Q_W1') #784,120
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]),name='Q_b1') #120

Q_w2_mu = tf.Variable(xavier_init([h_dim,z_dim]),name='Q_w2_mu') #120,100
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]),name='Q_b2_mu') #100

Q_w2_sigma = tf.Variable(xavier_init([h_dim,z_dim]),name='Q_w2_sigma') #120,100
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]),name='Q_b2_sigma')#100

In [9]:
print(X)

Tensor("X:0", shape=(?, 784), dtype=float32)


In [16]:
def Q(X):
    
    h = tf.nn.relu(tf.matmul(X,Q_w1) + Q_b1) # W.x+b
    z_mu = tf.matmul(h,Q_w2_mu) + Q_b2_mu 
    z_logvar = tf.matmul(h,Q_w2_sigma) + Q_b2_sigma
    
    return z_mu,z_logvar

In [13]:
def sample_z(mu,log_var):
    
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var/2) * eps

In [11]:
#P(X|z)
P_w1 = tf.Variable(xavier_init([z_dim,h_dim])) #100,120
P_b1 = tf.Variable(tf.zeros(shape=[h_dim])) #120

P_w2 = tf.Variable(xavier_init([h_dim,X_dim])) #120,784
P_b2 = tf.Variable(tf.zeros(shape=[X_dim])) #784

In [12]:
def P(z):
    
    h=tf.nn.relu(tf.matmul(z,P_w1) + P_b1)
    logits = tf.matmul(h,P_w2) + P_b2
    prob = tf.nn.sigmoid(logits)
    
    return prob,logits

In [24]:
#training phase starts here
z_mu,z_logvar = Q(X)
z_sample = sample_z(z_mu,z_logvar)
_,logits = P(z_sample)

#sample from random_z
X_samples,_ = P(z)

# E[log P(X|z)]
recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=X),1)

#D_KL(Q(Z|X) || P(Z)); calculate in closed form as both distribution are Gaussian
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar,1)

#VAE loss
vae_loss  = tf.reduce_mean(recon_loss + kl_loss)

#optimizer
solver = tf.train.AdamOptimizer().minimize(vae_loss)

#start the session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#make the output directory to save the image
if not os.path.exists('out/'):
    os.makedirs('out/')

#pointer
i=0

for it in range(1000000):
    #feed input image as batch_size
    X_mb,_ = mnist.train.next_batch(mb_size)
    
    _,loss = sess.run([solver,vae_loss],feed_dict={X:X_mb})
    
    if it % 1000 == 0:
        print("Iter:{}".format(it),end=" ")
        print("Loss:{:.4}".format(loss))
        print()
        
        samples = sess.run(X_samples,feed_dict={z:np.random.randn(16,z_dim)})
        
        fig = plot(samples)
        plt.savefig("out/{}.png".format(str(i).zfill(3)),bbox_inches='tight')
        i += 1
        
        plt.close(fig)

Iter:0 Loss:758.5

Iter:1000 Loss:145.1

Iter:2000 Loss:133.1

Iter:3000 Loss:122.0

Iter:4000 Loss:124.2

Iter:5000 Loss:112.6

Iter:6000 Loss:117.7

Iter:7000 Loss:110.8

Iter:8000 Loss:108.6

Iter:9000 Loss:105.5

Iter:10000 Loss:111.6

Iter:11000 Loss:104.6

Iter:12000 Loss:107.7

Iter:13000 Loss:111.4

Iter:14000 Loss:107.2

Iter:15000 Loss:114.5

Iter:16000 Loss:109.9

Iter:17000 Loss:108.1

Iter:18000 Loss:108.8

Iter:19000 Loss:113.1

Iter:20000 Loss:106.9

Iter:21000 Loss:104.7

Iter:22000 Loss:110.1

Iter:23000 Loss:108.9

Iter:24000 Loss:106.6

Iter:25000 Loss:111.3

Iter:26000 Loss:108.8

Iter:27000 Loss:110.3

Iter:28000 Loss:109.9

Iter:29000 Loss:107.6

Iter:30000 Loss:109.2

Iter:31000 Loss:112.1

Iter:32000 Loss:110.7

Iter:33000 Loss:108.2

Iter:34000 Loss:108.7

Iter:35000 Loss:114.2

Iter:36000 Loss:106.3

Iter:37000 Loss:110.8

Iter:38000 Loss:110.7

Iter:39000 Loss:110.2

Iter:40000 Loss:114.0

Iter:41000 Loss:107.1

Iter:42000 Loss:112.6

Iter:43000 Loss:103.4

I

Iter:347000 Loss:100.5

Iter:348000 Loss:102.3

Iter:349000 Loss:105.9

Iter:350000 Loss:106.6

Iter:351000 Loss:98.26

Iter:352000 Loss:110.3

Iter:353000 Loss:100.7

Iter:354000 Loss:107.6

Iter:355000 Loss:104.9

Iter:356000 Loss:101.7

Iter:357000 Loss:104.8

Iter:358000 Loss:99.24

Iter:359000 Loss:107.7

Iter:360000 Loss:103.2

Iter:361000 Loss:98.89

Iter:362000 Loss:117.5

Iter:363000 Loss:112.6

Iter:364000 Loss:106.0

Iter:365000 Loss:106.8

Iter:366000 Loss:107.8

Iter:367000 Loss:102.2

Iter:368000 Loss:106.6

Iter:369000 Loss:104.0

Iter:370000 Loss:108.7

Iter:371000 Loss:102.2

Iter:372000 Loss:106.2

Iter:373000 Loss:105.6

Iter:374000 Loss:108.7

Iter:375000 Loss:105.6

Iter:376000 Loss:99.39

Iter:377000 Loss:104.5

Iter:378000 Loss:108.5

Iter:379000 Loss:107.7

Iter:380000 Loss:109.4

Iter:381000 Loss:102.1

Iter:382000 Loss:107.9

Iter:383000 Loss:103.2

Iter:384000 Loss:100.2

Iter:385000 Loss:104.8

Iter:386000 Loss:104.3

Iter:387000 Loss:101.7

Iter:388000 Loss

Iter:689000 Loss:104.4

Iter:690000 Loss:107.5

Iter:691000 Loss:104.7

Iter:692000 Loss:97.79

Iter:693000 Loss:104.1

Iter:694000 Loss:100.9

Iter:695000 Loss:106.4

Iter:696000 Loss:106.1

Iter:697000 Loss:96.9

Iter:698000 Loss:104.7

Iter:699000 Loss:106.5

Iter:700000 Loss:104.6

Iter:701000 Loss:103.9

Iter:702000 Loss:106.2

Iter:703000 Loss:99.75

Iter:704000 Loss:104.7

Iter:705000 Loss:104.0

Iter:706000 Loss:102.9

Iter:707000 Loss:104.5

Iter:708000 Loss:99.91

Iter:709000 Loss:100.2

Iter:710000 Loss:101.0

Iter:711000 Loss:108.5

Iter:712000 Loss:105.2

Iter:713000 Loss:102.3

Iter:714000 Loss:101.9

Iter:715000 Loss:100.6

Iter:716000 Loss:97.38

Iter:717000 Loss:110.6

Iter:718000 Loss:102.1

Iter:719000 Loss:107.2

Iter:720000 Loss:104.9

Iter:721000 Loss:109.2

Iter:722000 Loss:108.1

Iter:723000 Loss:104.1

Iter:724000 Loss:96.11

Iter:725000 Loss:104.2

Iter:726000 Loss:102.9

Iter:727000 Loss:105.8

Iter:728000 Loss:97.95

Iter:729000 Loss:104.8

Iter:730000 Loss: